In [19]:
import pandas as pd
import matplotlib.pyplot as plt
#read data
df = pd.read_csv('../data/curated/growth_rate3.csv')
df.head()

,post_code,growth_rate
0,3714,2.267857
1,3779,1.077143
2,3019,0.793766
3,3075,0.446740
4,3026,0.409205


In [20]:
import geopandas as gpd
import folium
import seaborn as sns
sf = gpd.read_file("../data/raw/VMADMIN/POSTCODE_POLYGON.shp")
sf['centroid'] = sf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
# create a JSON 
postcode = []
for i in sf.iloc[:,1:2].values:
    postcode.append(int(i))
sf['postcode'] = postcode
geoJSON = sf[['postcode', 'geometry']].drop_duplicates('postcode').to_json()

In [21]:
# top 10 suburbs with highest growth rates 
df_top10_growth_rate = df.sort_values(by='growth_rate', axis=0, ascending=False).head(10)
df_top10_growth_rate

,post_code,growth_rate
0,3714,2.267857
1,3779,1.077143
2,3019,0.793766
3,3075,0.446740
4,3026,0.409205
5,3047,0.400249
6,3020,0.356444
7,3048,0.339016
8,3060,0.323021
9,3038,0.321212


In [22]:
#merge the growth rate dataset with the postcode polygon dataset
post_code = []
for i in sf['POSTCODE']:
    post_code.append(int(str(i)))
sf['post_code'] = post_code
df_top10 = pd.merge(df_top10_growth_rate,sf,on = ['post_code'],how='left')

In [23]:

m3 = folium.Map(location=[-37.0,144], control_scale=True, zoom_start=7)

# create the map showing the predicted rate of all the postcodes
c3 = folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth2', 
    data=df, 
    columns=['post_code','growth_rate'],
    key_on='properties.postcode',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name='growth_rate'
)

c3.add_to(m3)
df_copy = df_top10.copy()

#adding the green marker of top 10 suburbs
for i in range(0, len(df_copy)):
    html="""
    <h4> Postcode: </h4>""" + str(df_copy.iloc[i]['post_code'])+ \
    """<h4>Growth_rate:</h4>""" + str(df_copy.iloc[i]['growth_rate'])
    iframe = folium.IFrame(html=html, width=200, height=300)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[df_copy.iloc[i]['centroid'][0],df_copy.iloc[i]['centroid'][1]],
        popup= popup,
        tooltip=str(df_copy.iloc[i]['post_code']),
        icon=folium.Icon(color='green',icon='home',prefix="fa"),
    ).add_to(m3)

In [24]:
#save the visualization of top 10 suburbs with highest pridicted grow rate in plots
import os
fig_save_path = "../plots/"
if not os.path.exists(fig_save_path):
    os.makedirs(fig_save_path)
m3.save(os.path.join(fig_save_path,"top 10 suburbs with the highest predicted growth rate.html"))